# *CoastSat*: Multi Beach Downloader


## Initial settings

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects
import ee
ee.Initialize(project='ee-rafarencoretg')

## 1. Retrieval of the images from GEE

Se crea un diccionario con la información de las playas de la región a descargar.

In [2]:
country_info = {
    "country": "CHILE",
    "country_code": "chi",
    "region": {
        "OHIGGINS": {
            "chi0345": {
                "name": "PLAYA_MATANZAS",
                "slope": 0.074
            },
            "chi0356": {
                "name": "PLAYA_PUNTA_DE_LOBOS",
                "slope": 0.075
            }
        }
    }
}

Inicialización de los inputs del código, tales como fechas, región, códigos, pendientes y satélites. Se realiza un chequeo previo a la descarga.

In [6]:
# Sitio de estudio y su codigo
region_name = "OHIGGINS"
geojson_folder = 'CoastSat_Web'

# Rango de fechas
dates = ['2024-11-01', '2024-12-31']

# Satelites (misiones) ['L5','L7','L8','L9','S2']
sat_list = ['L8','L9','S2']

# Directorio de almacenamiento
filepath = os.path.join(os.getcwd(), 'data')

for beach in country_info["region"][region_name]:
    sitename = country_info["region"][region_name][beach]["name"]
    slope = country_info["region"][region_name][beach]["slope"]

    # Region de interes (longitud, latitud) en formato geojson
    geojson_polygon = os.path.join(os.getcwd(), geojson_folder, region_name, beach, f"{beach}_polygon.geojson")
    polygon = SDS_tools.polygon_from_geojson(geojson_polygon)

    # Diccionario con la información del sitio de estudio
    inputs = {'region': region_name, 'id': beach, 'sitename': sitename, 'slope': slope, 
              'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'filepath':filepath}

    # Chequeo de cantidad de imagenes disponibles
    print(f'\nIMAGENES DISPONIBLES EN {sitename}')
    SDS_download.check_images_available(inputs);


IMÁGENES DISPONIBLES EN PLAYA MATANZAS
Number of images available between 2024-11-01 and 2024-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L8: 12 images
     L9: 11 images
     S2: 22 images
  Total to download: 45 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L8: 8 images
  Total Tier 2: 8 images

IMÁGENES DISPONIBLES EN PLAYA PUNTA DE LOBOS
Number of images available between 2024-11-01 and 2024-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L8: 8 images
     L9: 7 images
     S2: 33 images
  Total to download: 48 images
L8: 8/8 images already exist, 0 to be downloaded
L9: 7/7 images already exist, 0 to be downloaded
S2: 33/33 images already exist, 0 to be downloaded
- In Landsat Tier 2 (not suitable for time-series analysis):
     L8: 4 images
  Total Tier 2: 4 images


Luego del chequeo, se descargan las imágenes.

In [10]:
for beach in country_info["region"][region_name]:
    sitename = country_info["region"][region_name][beach]["name"]
    slope = country_info["region"][region_name][beach]["slope"]

    # Region de interes (longitud, latitud) en formato geojson
    geojson_polygon = os.path.join(os.getcwd(), geojson_folder, region_name, beach, f"{beach}_polygon.geojson")
    polygon = SDS_tools.polygon_from_geojson(geojson_polygon)

    # Diccionario con la información del sitio de estudio
    inputs = {'region': region_name, 'id': beach, 'sitename': sitename, 'slope': slope, 
              'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'filepath':filepath}

    # Descarga de imagenes
    print(f"\nDESCARGANDO IMAGENES EN {sitename}")
    metadata = SDS_download.retrieve_images(inputs)


DESCARGANDO IMAGENES EN PLAYA MATANZAS
GEE initialized (existing token).
Number of images available between 2024-11-01 and 2024-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L8: 12 images
     L9: 11 images
     S2: 22 images
  Total to download: 45 images
L8: 12/12 images already exist, 0 to be downloaded
L9: 11/11 images already exist, 0 to be downloaded
S2: 22/22 images already exist, 0 to be downloaded
- In Landsat Tier 2 (not suitable for time-series analysis):
     L8: 8 images
  Total Tier 2: 8 images

L8: 0 images

L9: 0 images

S2: 0 images

Satellite images downloaded from GEE and save in c:\Users\pedro\Documents\RAFA\CoastSat-RRG\data\PLAYA MATANZAS

DESCARGANDO IMAGENES EN PLAYA PUNTA DE LOBOS
GEE initialized (existing token).
Number of images available between 2024-11-01 and 2024-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L8: 8 images
     L9: 7 images
     S2: 33 images
  Total to download: 48 images
L8: 8/8 images already exist, 0 to be downloaded
L

Finalmente, se construye la metadata de los sitios de estudio.

In [11]:
for beach in country_info["region"][region_name]:
    sitename = country_info["region"][region_name][beach]["name"]
    slope = country_info["region"][region_name][beach]["slope"]

    # Region de interes (longitud, latitud) en formato geojson
    geojson_polygon = os.path.join(os.getcwd(), geojson_folder, region_name, beach, f"{beach}_polygon.geojson")
    polygon = SDS_tools.polygon_from_geojson(geojson_polygon)

    # Diccionario con la información del sitio de estudio
    inputs = {'region': region_name, 'id': beach, 'sitename': sitename, 'slope': slope, 
              'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'filepath':filepath}
    
    # Grabado de metadata
    print(f'\nGRABADO DE METADATA DE {sitename} COMPLETADA')
    metadata = SDS_download.get_metadata(inputs)


GRABADO DE METADATA DE PLAYA MATANZAS COMPLETADA

GRABADO DE METADATA DE PLAYA PUNTA DE LOBOS COMPLETADA
